In [1]:
import os
import base64
import hashlib
from Crypto.Cipher import AES
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Hash import MD5


In [2]:
# WRITING TO FILES
def savePrivateKey(private, size):
    with open("privateKey.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tPrivate key\n\n")
        f.write("Method:\n")
        f.write("\tRSA\n\n")
        f.write("Key length:\n")
        f.write("\t0{0:x}".format(size)+"\n")
        f.write("\nPrivate key: \n")
        f.write(str(private.exportKey())[2:-1])
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")


def savePublicKey(public, size):
    with open("publicKey.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tPublic key\n\n")
        f.write("Method:\n")
        f.write("\tRSA\n\n")
        f.write("Key length:\n")
        f.write("\t0{0:x}".format(size)+"\n")
        f.write("\nPublic key: \n")
        f.write(str(public.exportKey())[2:-1])
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")


def saveAESKey(key):
    with open("aesKey.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tSecret key\n\n")
        f.write("Method:\n")
        f.write("\tAES\n")
        f.write("\nSecret key: \n")
        f.write(str(key)[2:-1])
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")

def printEnvelope(data, encryptedKey, aes_size, rsa_size):
    with open("envelope.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tEnvelope\n\n")
        f.write("Method:\n")
        f.write("\tAES\n")
        f.write("\tRSA\n")
        f.write("Key length:\n")
        f.write("\t0{0:x}".format(aes_size) + "\n")
        f.write("\t0{0:x}".format(rsa_size) + "\n")
        f.write("\nEnvelope data: \n")
        f.write(str(data))
        f.write("\n")
        f.write("\nEnvelope crypt key: \n")
        f.write(str(encryptedKey)[2:-2])
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")


def printSignature(digest, sha_size, rsa_size):
    with open("signature.txt", "w") as f:
        f.write("---BEGIN OS2 CRYPTO DATA---\n")
        f.write("\nDescription:\n")
        f.write("\tSignature\n\n")
        f.write("Method:\n")
        f.write("\tSHA-1\n")
        f.write("\tRSA\n")
        f.write("\nKey length:\n")
        f.write("\t0{0:x}".format(sha_size) + "\n")
        f.write("\t0{0:x}".format(rsa_size) + "\n")
        f.write("\nSignature: \n")
        f.write(str(digest))
        f.write("\n")
        f.write("\n---END OS2 CRYPTO DATA---")

In [3]:
# AES
EncodeAES = lambda c, s: base64.b64encode(c.encrypt(s))
DecodeAES = lambda c, e: c.decrypt(base64.b64decode(e))

# generate a random secret key and initialization vector
secret = os.urandom(16)
IV = os.urandom(16)
print ("Secret key:", str(secret)[2:-1])
saveAESKey(secret)

cipher = AES.new(secret, AES.MODE_CFB, IV)

encoded = EncodeAES(cipher, 'test string')
print ('Encrypted string:', str(encoded)[2:-1])

cipher = AES.new(secret, AES.MODE_CFB, IV)

decoded = DecodeAES(cipher, encoded)
print ('Decrypted string:', str(decoded)[2:-1])

Secret key: \xc9]J\xd5\xa0\xe3o\xb3\x8aI\xf3<\xbeD\xfe4
Encrypted string: l/uQWTgcYYcOXZI=
Decrypted string: test string


In [4]:
# RSA

KEY_LENGTH = 1024
random_gen = Random.new().read
 
keypair = RSA.generate(KEY_LENGTH, random_gen)
 
publicKey = keypair.publickey()
savePublicKey(publicKey, 1024)
savePrivateKey(keypair, 1024)
 
print("AES key:", str(secret)[2:-1], "\n")
message = secret

messageHash = MD5.new(message).digest()

signature = keypair.sign(messageHash, '')
 
encryptedMessage = publicKey.encrypt(message, 32)
print ("Encrypted AES key: ", str(encryptedMessage)[3:-3], "\n")
 
decryptedMessage = keypair.decrypt(encryptedMessage)
 
# Signature validation
decryptedHash = MD5.new(decryptedMessage).digest()

if publicKey.verify(decryptedHash, signature):
    print ("Decrypted AES key:", str(decryptedMessage)[2:-1], "\n")

AES key: \xc9]J\xd5\xa0\xe3o\xb3\x8aI\xf3<\xbeD\xfe4 

Encrypted AES key:  \xd4\x02`\xf9\xfe\x95M\xf6\x86\xd1mU\xa4\xceV}\xa0\x9f>\xd6\xc9\xb1\x9f\xa7 \x89a\rL\x8d\x8b\xcc\xaaV\x10aWt_!\xf1f\x0b\x06\x97\xa2\xda\xdf\xbd`\\\x1e\xbbY_\xf1a\xc9\xe3\x94?\x17}\xab\xfcp\xb7\xba\xde\xaa\xa7!\x9a\xdd\xec-\x13D\xc6S\xde\x82Q\x8f\x9f;\xb2\x03\xa0\xe0F@\x02SG\xb9,&\xbc\xec2q{\xbea\x82m\xa4t\xe7xho}\x7f$_\xb5\xbeW\xcf\xbc\x08\xc7\xcb\xab\x1e\xde 

Decrypted AES key: \xc9]J\xd5\xa0\xe3o\xb3\x8aI\xf3<\xbeD\xfe4 



In [5]:
# DIGITALNA OMOTNICA

# KRIPTIRANJE
P = "Ovo je moja poruka"
print("Pocetna poruka: ", P, "\n")

AES_BLOCK_SIZE = 16

secret = os.urandom(AES_BLOCK_SIZE)
IV = os.urandom(AES_BLOCK_SIZE)

cipher = AES.new(secret, AES.MODE_CFB, IV)

C1 = EncodeAES(cipher, P)

keypair = RSA.generate(KEY_LENGTH, random_gen)
publicKey = keypair.publickey()

C2 = publicKey.encrypt(secret, 32)

M = (C1, C2)

print("Sadrzaj omotnice M:\n", M, "\n")

printEnvelope(C1, C2, 128, 1024)

# DEKRIPTIRANJE

key = keypair.decrypt(C2)

cipher = AES.new(key, AES.MODE_CFB, IV)

P = DecodeAES(cipher, C1)
print ('Dekriptirana poruka:', P.decode('ascii'))

Pocetna poruka:  Ovo je moja poruka 

Sadrzaj omotnice M:
 (b'fQChJTfDYNt3BE5Szw5m44+C', (b'\x89|Vj\x06\xe7\xf5\xf3\xd6\x9c\x88k\r\x8f\x9a\t\xf4\x1eY<t\xfc\xbb\xc4\x18\xc3.L\x8e~\xf4\xf2\xef\xb4\xc4\x97\x1f\x12\xc3v0\x18(1\x19^\x19\xa0(\xe8\xb8v\xb0\xb4z\x14\xd1\xa2\xbbd\xf8\xf8r\xa1\xf3*Zv\x9f\xed\x87\x06\x17hl+<2\xa7\x05\x8d\xc4_\xf8\xbce\xef\x19\x14\xe3\x8dU\xb4\xe5\xb2\xfej\xba/2\x91v\xed\x14\xcf3X\x8bd2\xe24\x04\xad+\x9diE\xcc\x82S\xf3P\xba?Ci[',)) 

Dekriptirana poruka: Ovo je moja poruka


In [6]:
# DIGITALNI POTPIS

m = hashlib.sha1()
keypair = RSA.generate(KEY_LENGTH, random_gen)
publicKey = keypair.publickey()

P = "ovo je poruka".encode('utf-8')

m.update(P)
H = m.digest()
hexdigest = m.hexdigest()

encrypted_digest = publicKey.encrypt(H, 32)

M = (P, encrypted_digest)

print ("M = ", M, "\n")

decrypted_digest = keypair.decrypt(encrypted_digest)

printSignature(hexdigest, 160, 1024)

print("Digest:", str(H)[2:-1])
print("Hex digest:", hexdigest)
print("Decrypted digest:", str(decrypted_digest)[2:-1])

M =  (b'ovo je poruka', (b"nq\\\xd5\xe5\x80\xe6\xff\x95\xe1D\xecI\xe3\t\x1e\x96'\n\x9f\x17s\xed\xe9YM\x87\x91\x97\xf1\xe1\xd9\x9a\xadI\xa6l6\x9b\x82L\xff\xb5X\x82\x17\xfa^\x01pG\x8bc\xfd\xe4`\xb5&C2\x14\x7f\xe1\xe03\xec8[\xa3\x81\x93\xdb\xe0\xa6K\x9c\x81\x92]\x87\x9fK\x056\x8eXe\xaf2 685\x1d\x0c\x89\x17\x9c\x0c@\xb2\xcc\xc9LC\xdc\x1c\xbavh@\x07\xfa\xde0\xd7\xb4f%\x83\x1d\x98h\xb9\xc4\xefK\xa3",)) 

Digest: S\xf8\xa01\xc8\xe5\xc3\x9c\x93\xb7?\xa5\x11\x8f(v\x9ai\x92\xd0
Hex digest: 53f8a031c8e5c39c93b73fa5118f28769a6992d0
Decrypted digest: S\xf8\xa01\xc8\xe5\xc3\x9c\x93\xb7?\xa5\x11\x8f(v\x9ai\x92\xd0


In [7]:
# DIGITALNI PECAT

keypair = RSA.generate(KEY_LENGTH, random_gen)
publicKey = keypair.publickey()
secret = os.urandom(AES_BLOCK_SIZE)
IV = os.urandom(16)

cipher = AES.new(secret, AES.MODE_CFB, IV)
m = hashlib.sha1(P)

C1 = EncodeAES(cipher, "ovo je poruka")
C2 = publicKey.encrypt(secret, 32)
C3 = publicKey.encrypt(H, 32)

M = (C1, C2, C3)

print("M(C1, C2, C3):", M, "\n")

K = keypair.decrypt(C2)
cipher = AES.new(K, AES.MODE_CFB, IV)
P = DecodeAES(cipher, C1)
S = keypair.decrypt(C3)

print ("K =", str(K)[2:-1])
print ("P =", str(P)[2:-1])
print ("S =", str(S)[2:-1])

m = hashlib.sha1()
m.update(P)
h = m.digest()

if (h == S):
    print ("\nPrimljeni i izracunati sazetak se podudaraju!")

M(C1, C2, C3): (b'9x/PO35rpyUoPxlU6w==', (b"\x88\x01\xa7\xea\xe3G\x9dh\xd5-\xa6\x02\n,\xcc\xde\xc3\x8f\xf6\xa9\xfe\xd6\x06\x03g0\x93\xdb\xb9\xda<3\x8c?V0=#WS\x84\xb3\xbb\xc7\xcfq\xd8p\xe2\xd5\xa0tX\xc3\x12\xe7\xcbE\x83\xe9Z\xf0\x1cZ\xdbh\x99\xb8\xfb\xee\x0b#\xf9\xf9\x1aa@XO\xe6\x16\xfe#\x13\xa5\xb3\xa8O>\xf8\xc3\xc4\xa2r\xc5\xc3\x99U\x859*y\xb6\xf9\x8bR\xd8>\xf7\xc2\x91\x8ae\x9b0\xa5\x86\x83H\xb4<\x1e\x0e\x95S'\x85\xd6",), (b'\x81\x14\x9f\xc0#\xf8\xa54\t\xfb8\xf0\x11\x9e\xa3rN\xc9\x80\xf2NqQT\xf6c\xad\xf5\x9bsc\xca\xaa\xaesx\x12\xc1QN\xcf#j5Sq\xab\xe1\xb4p\x9c\xa5\xc6\x93*>*^\x0b\xf8\xfc\xa1\xe5\xa5$\xc3]\xb6f\x16\xfe\xdd\x16\xae<\xca\xc6]\x1814O\xd7\xc1\xe3\xc9\x04\x8f\xcd>\xc1;\x1cN\xebk\x0bt\xe3o\xd0\xc9\x9d\xe8\x19o\xa8\xde\xbd\xe9\xf3u\xd7^\xec\xc8\x8b\x82\x1b\xeb\xfb\xc34)\x8f\xc3\xcb\xa8',)) 

K = \n\xc9\xe5\xc9\xd9s\xf4\x1a\xb2\xd9\x15I\xf3\xecg+
P = ovo je poruka
S = S\xf8\xa01\xc8\xe5\xc3\x9c\x93\xb7?\xa5\x11\x8f(v\x9ai\x92\xd0

Primljeni i izracunati sazetak se podudaraju!


In [8]:
# SHA-1 IMPLEMENTATION

"""
Razbija poruke u komade od n bitova.
"""
def chunks(array, n):
        return [array[i:i+n] for i in range(0, len(array), n)]

"""
Rotacija dobivenog niza u lijevo za n bitova.
"""
def rotate_left(word, n):
    return ((word << n) | (word >> (32 - n))) & 0xffffffff
    
def sha1(data):
    bytes = ""
    
    # INICIJALIZACIJA VARIJABLE
    h0 = 0x67452301
    h1 = 0xEFCDAB89
    h2 = 0x98BADCFE
    h3 = 0x10325476
    h4 = 0xC3D2E1F0
    
    # PRED-OBRADA
    
    # pretvaranje ulaznog stringa u 8-bitni zapis
    for n in range(len(data)):
        bytes += '{0:08b}'.format(ord(data[n]))
    
    # dodavanje 1 na kraj poruke
    bits = bytes + "1"

    paddedBits = bits
    
    # dodavanje k 0 sve dok rezultirajuca poruke bude kongruent 448 (mod 512)
    while len(paddedBits) % 512 != 448:
        paddedBits += "0"
        
    # dodaj duljinu pocetnog niza kao 64-bitni big-endidan cijeli broj
    paddedBits += '{0:064b}'.format(len(bits) - 1)
    
    # PODJELA NA DIJELOVE I OBRADA
    for c in chunks(paddedBits, 512): 
        words = chunks(c, 32)
        w = [0]*80

        for n in range(0, 16):
            w[n] = int(words[n], 2)
            
        for i in range(16, 80):
            w[i] = rotate_left((w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16]), 1)  

        a = h0
        b = h1
        c = h2
        d = h3
        e = h4

        for i in range(0, 80):
            if 0 <= i <= 19:
                f = (b & c) | ((~b) & d)
                k = 0x5A827999
            elif 20 <= i <= 39:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif 40 <= i <= 59:
                f = (b & c) | (b & d) | (c & d) 
                k = 0x8F1BBCDC
            elif 60 <= i <= 79:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = rotate_left(a, 5) + f + e + k + w[i] & 0xffffffff
            
            e = d
            d = c
            c = rotate_left(b, 30)
            b = a
            a = temp

        h0 = h0 + a & 0xffffffff
        h1 = h1 + b & 0xffffffff
        h2 = h2 + c & 0xffffffff
        h3 = h3 + d & 0xffffffff
        h4 = h4 + e & 0xffffffff

    return '%08x%08x%08x%08x%08x' % (h0, h1, h2, h3, h4)

In [9]:
m = hashlib.sha1()
m.update("ovo je test string koji sigurno nije visekratnik 100% definitivno".encode('utf-8'))
print(m.hexdigest())
print(sha1("ovo je test string koji sigurno nije visekratnik 100% definitivno"))

ee492e99aeb1caf267861a496e4edce6f8409c98
ee492e99aeb1caf267861a496e4edce6f8409c98
